In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

fileList = []
indx = 0
th = 9

# Load the .fasta file into a dataframe

with open('sample.fasta', 'r') as file:
    for line in file:
        if line.strip()[0] == '>':
            entry = [[],'25-III','B','S',[]]
            fileList.append(entry)
            fileList[indx][0] =  line.strip()
            indx += 1
        else:
            fileList[indx-1][4] += line.strip()
df = pd.DataFrame(fileList, columns=["ID","DATE","SPC","AMB","SEQ"])


In [112]:
df

,ID,DATE,SPC,AMB,SEQ
0,>NG_048059.1 Enterobacter cloacae 115-824A rmt...,25-III,B,S,"[A, T, G, A, A, C, A, T, C, A, A, C, G, A, T, ..."
1,>OM212032.1 Escherichia coli strain CMCY-6 16S...,25-III,B,S,"[A, T, G, G, A, T, A, A, G, A, A, T, G, A, T, ..."
2,>NG_047476.1 Klebsiella pneumoniae BM4536 pIP1...,25-III,B,S,"[A, A, C, A, G, T, C, G, T, A, G, T, T, C, G, ..."
3,>NG_048058.1 Serratia marcescens S-95 rmtB gen...,25-III,B,S,"[T, A, G, G, T, G, A, A, G, A, T, C, C, T, T, ..."


In [113]:
def compSeq(seq1, seq2, shift):
    l = 0
    if(len(seq1) < len(seq2)):
        l = len(seq1)
    else:
        l = len(seq2)
        app = seq1
        seq1 = seq2
        seq2 = app
    
    mVal = np.zeros(len(seq2)-len(seq1) + 2*shift)

    for s in range(len(seq2)-len(seq1) + 2*shift):
        nMatch = 0
        lMatch = 0
        hLenMatch = np.zeros(len(seq1) - shift)

        app1 = []
        app2 = []

        lScan = 0

        if(s < shift):
            app1 = seq1[shift-s:len(seq1)]
            app2 = seq2[0:len(seq1)-(shift-s)]
            lScan = len(seq1)-(shift-s)
        elif(s > len(seq2) - len(seq1)):
            app1 = seq1[0:  len(seq2) - s]
            app2 = seq2[s:len(seq2)]
            lScan = len(seq2) - s
        else:
            app1 = seq1
            app2 = seq2[s:len(seq1)+s]
            lScan = len(seq1)
        
        for i in range(lScan):
            if(app1[i] == app2[i]):
                nMatch += 1
                lMatch += 1
            else:
                hLenMatch[lMatch] += 1
                lMatch = 0
        
        mVal[s] = nMatch/lScan
    
    return mVal




In [114]:
def findMax(seq1, seq2):

    if(len(seq1) > len(seq2)):
        app = seq1
        seq1 = seq2
        seq2 = app

    lSeq = th
    maxSeq = []
    maxs = 0
    maxk = 0
    exists = True

    while lSeq < len(seq1) and exists:
        s = 0
        exists = False
        while s < len(seq1) - lSeq:
            k = 0
            app1 = seq1[s:lSeq+s]
            while k < len(seq2) - lSeq:
                app2 = seq2[k:lSeq+k]
                while app1 == app2:
                    maxSeq = app1
                    app1 = seq1[s:lSeq+s]
                    app2 = seq2[k:lSeq+k]
                    maxs = s
                    maxk = k
                    lSeq += 1
                    exists = True
                k += 1
            s += 1
        lSeq += 1
    
    return maxSeq,maxs,maxk
        

In [ ]:
n = 4
for i in range(n):
    for j in range(i+1,n):
        seq_1 = list(df.at[i,'SEQ'])
        seq_2 = list(df.at[j,'SEQ'])
        mask_1 = np.zeros(len(seq_2))
        mask_2 = np.zeros(len(seq_2))
        maskId = 1

        corr = False

        commonSeq,idxi,idxj = findMax(seq_1,seq_2)
        while(len(commonSeq) > th):
            corr = True
            print(commonSeq)
            for k in range(len(commonSeq) - 1):
                seq_1[idxi + k] = 0
                seq_2[idxj + k] = 1
                mask_1[idxi + k] = maskId
                mask_2[idxj + k] = maskId
            maskId += 1
            commonSeq,idxi,idxj = findMax(seq_1,seq_2)
        if corr:
            fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10,2))
            ax[0].pcolormesh([mask_1]*2, cmap='tab20', shading='gouraud')
            ax[1].pcolormesh([mask_2]*2, cmap='tab20', shading='gouraud')

        plt.show()



['G', 'A', 'C', 'T', 'G', 'G', 'C', 'C', 'G', 'A', 'G', 'C', 'T', 'G', 'G', 'A', 'T', 'A', 'C', 'C', 'C', 'T', 'G', 'T', 'A', 'C', 'G', 'A', 'T', 'T', 'T', 'T', 'A', 'T', 'C', 'T', 'T', 'T', 'T', 'C', 'A', 'G', 'C', 'C', 'G', 'A', 'A', 'A', 'C', 'T', 'C', 'C', 'C', 'C', 'G', 'C', 'C', 'G', 'C', 'G', 'T', 'G', 'C', 'T', 'G', 'G', 'A', 'T', 'A', 'T', 'C', 'G', 'C', 'C', 'T', 'G', 'C', 'G', 'G', 'T', 'C', 'T', 'T', 'A', 'A', 'C', 'C', 'C', 'C', 'T', 'T', 'G', 'G', 'C', 'G', 'C', 'T', 'A', 'T', 'A', 'C', 'G', 'A', 'G', 'C', 'G', 'C', 'G', 'G']
['A', 'T', 'C', 'A', 'C', 'C', 'C', 'C', 'C', 'T', 'T', 'T', 'G', 'C', 'T', 'A', 'G', 'G', 'G', 'A', 'A', 'A', 'A', 'A', 'G', 'A', 'T', 'T', 'G', 'G', 'G', 'A', 'T', 'T', 'T', 'T', 'A', 'C', 'C', 'T', 'T', 'T', 'G', 'C', 'C', 'C', 'T', 'G', 'C', 'A', 'G', 'G', 'A', 'T', 'G', 'T', 'G', 'C', 'T', 'G', 'T', 'G', 'T', 'G', 'C', 'G', 'C', 'C', 'G', 'C', 'C', 'C', 'G', 'C', 'C', 'G', 'A', 'A']
['A', 'T', 'G', 'A', 'A', 'C', 'A', 'T', 'C', 'A', 'A', 'C', 'G

AttributeError: Figure.set() got an unexpected keyword argument 'size'